In [4]:
using StreamOps

In [2]:
function run_test()

    pipe = @streamops EWMean{Float64}(; alpha=0.1) Diff{Float64}(1) Func(x -> x + 1)
    # pipe = OpFunc(x -> x + 1; next=OpEWMean{Float64}(0.1)) 

    pipe(1.0)

    @time for i in 1.0:1_000_000.0
        pipe(i)
    end

    pipe(1.0)
end

run_test()

  0.004995 seconds


-99998.0

In [3]:
@macroexpand @streamops Func(x -> x + 1) EWMean{Float64}(; alpha=0.1)

quote
    #= /Users/rino/repos/StreamOps.jl/src/macros.jl:291 =#
    local var"#78###state#231" = Func((x->begin
                        #= /Users/rino/repos/StreamOps.jl/test_ops.ipynb:1 =#
                        x + 1
                    end))
    local var"#79###state#233" = EWMean{Float64}(; alpha = 0.1)
    #= /Users/rino/repos/StreamOps.jl/src/macros.jl:292 =#
    function (var"#82#x"::var"#83#T",) where var"#83#T"
        #= /Users/rino/repos/StreamOps.jl/src/macros.jl:292 =#
        #= /Users/rino/repos/StreamOps.jl/src/macros.jl:293 =#
        begin
            local var"#80###call#232" = var"#78###state#231"(var"#82#x")
            local var"#81###call#234" = var"#79###state#233"(var"#80###call#232")
        end
    end
end

In [9]:
using BenchmarkTools

function test_manual(trials)
    last::Ref{Float64} = Ref(Float64(NaN))
    pipeline = function (x)
        x = abs(x)
        x = x * x
        tmp = last[]
        last[] = x
        tmp
    end
    current::Float64 = Float64(NaN)
    for i in 1:trials
        current = pipeline(Float64(i))
    end
    current
end

function test_pipeline(count)
	pipeline = @streamops Prev{Float64}
	current::Float64 = Float64(NaN)
	for i in 1:count
		current = pipeline(Float64(i))
	end
	current
end
trials = 10_000_000
println("Trials: $trials")
display(@benchmark test_manual(trials) samples = 30 evals = 3)
display(@benchmark test_pipeline(trials) samples = 30 evals = 3)

Expr
  head: Symbol curly
  args: Array{Any}((2,))
    1: Symbol Prev
    2: Symbol Float64


LoadError: LoadError: Unkown operation: Prev{Float64}
in expression starting at /Users/rino/repos/StreamOps.jl/test_ops.ipynb:20

In [8]:
# using ProfileView
# ProfileView.@profview map(x -> test(1e7), 1:10)

# using Profile, ProfileSVG
# @Profile.profile test_pipeline(1e8)
# ProfileSVG.view(maxdepth=300, maxframes=3000)